In [179]:
from langchain.tools import tool
from langchain_core.tools import InjectedToolArg
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import HumanMessage
from langchain_openai import OpenAI
from typing import Annotated
import requests
from dotenv import load_dotenv
load_dotenv()
import os


In [252]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> str:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: str, conversion_rate: Annotated[str, InjectedToolArg]) -> str:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return str(float(base_currency_value) * float(conversion_rate))


In [248]:
llm=ChatMistralAI(api_key=os.getenv('Mistral'),model='mistral-large-latest')
llm_tool=llm.bind_tools(tools=[get_conversion_factor,convert])
llm_tool.as_tool


<bound method Runnable.as_tool of RunnableBinding(bound=ChatMistralAI(client=<httpx.Client object at 0x00000276D8FA8950>, async_client=<httpx.AsyncClient object at 0x00000276D8FAA630>, mistral_api_key=SecretStr('**********'), endpoint='https://api.mistral.ai/v1', model='mistral-large-latest', model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculates the target currency value from a given base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'string'}}, 'required': ['base_currency_value'], 'ty

In [268]:
messages = [HumanMessage('get the conversion rate from 100 inr to usd')]

In [269]:
ai_message=llm_tool.invoke(messages)
messages.append(ai_message)


In [270]:
print(messages)

[HumanMessage(content='get the conversion rate from 100 inr to usd', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7QE3dzo0X', 'function': {'name': 'get_conversion_factor', 'arguments': '{"base_currency": "inr", "target_currency": "usd"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 190, 'total_tokens': 227, 'completion_tokens': 37}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run--5783a276-b042-48a2-bec4-21802f7e0fee-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'inr', 'target_currency': 'usd'}, 'id': '7QE3dzo0X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 37, 'total_tokens': 227})]


In [271]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'inr', 'target_currency': 'usd'},
  'id': '7QE3dzo0X',
  'type': 'tool_call'}]

In [272]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
second_tool_message = convert.invoke({'base_currency_value': '1000', 'conversion_rate': str(conversion_rate)})
messages.append(second_tool_message)



In [273]:
messages

[HumanMessage(content='get the conversion rate from 100 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7QE3dzo0X', 'function': {'name': 'get_conversion_factor', 'arguments': '{"base_currency": "inr", "target_currency": "usd"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 190, 'total_tokens': 227, 'completion_tokens': 37}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run--5783a276-b042-48a2-bec4-21802f7e0fee-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'inr', 'target_currency': 'usd'}, 'id': '7QE3dzo0X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 37, 'total_tokens': 227}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 175279680

In [274]:
llm.invoke(str(messages))

AIMessage(content='The conversion rate from INR to USD is 0.01162. Therefore, 100 INR is equivalent to approximately 1.16 USD.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 565, 'total_tokens': 603, 'completion_tokens': 38}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run--2bd34810-746f-4a10-820b-95889a0da2c8-0', usage_metadata={'input_tokens': 565, 'output_tokens': 38, 'total_tokens': 603})